<a href="https://colab.research.google.com/github/TeachingTextMining/TextClassification/blob/main/01-SA-Pipeline-Reviews/01-SA-Pipeline-Reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Entrenamiento y ejecución de un pipeline de clasificación textual.

La clasificación de textos consiste en, dado un texto, asignarle una entre varias categorías. Algunos ejemplos de esta tarea son:

- dado un tweet, categorizar su connotación como positiva, negativa o neutra.
- dado un post de Facebook, clasificarlo como portador de un lenguaje ofensivo o no.  

En la actividad exploraremos cómo crear un pipeline y entrenarlo para clasificar reviews de [IMDB]() sobre películas en las categorías \[$positive$, $negative$\]

Puede encontrar más información sobre este problema en [Kaggle](https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews) y en [Large Movie Review Datase](http://ai.stanford.edu/~amaas/data/sentiment/). 

**Instrucciones:**

- siga las indicaciones y comentarios en cada apartado.

**Después de esta actividad nos habremos familiarizado con:**
- algunos tipos de carácterísticas ampliamente utilizadas en la clasificación de textos. 
- cómo contruir un pipeline para la clasificación de textos utilizando sklearn.
- utilizar este pipeline para clasificar nuevos textos.

**Requerimientos**
- python 3.6 - 3.8
- pandas
- plotly


### Instalación de librerías e importación de dependencias.

Para comenzar, es preciso instalar e incluir las librerías necesarias. En este caso, el entorno de Colab incluye las necesarias.

Ejecute la siguiente casilla prestando atención a las explicaciónes dadas en los comentarios.

In [ ]:
#  para construir gráficas y realizar análisis exploratorio de los datos
import plotly.graph_objects as go

# para cargar datos y realizar pre-procesamiento básico
import pandas as pd
from collections import Counter

# para pre-procesamiento del texto y extraer carácterísticas
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.stem.snowball import EnglishStemmer

# algoritmos de clasificación
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

# para construir pipelines
from sklearn.pipeline import Pipeline

# para evaluar los modelos 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay, roc_auc_score, plot_roc_curve
from sklearn.utils.multiclass import unique_labels

# para guardar el modelo
import pickle

print('Done!')

### Definición de funciones y variables necesarias para el preprocesamiento de datos

Antes de definir el pipeline definiremos algunas variables útiles como el listado de stop words y funciones para cargar los datos, entrenar el modelo etc.

In [ ]:
#listado de stopwords. Este listado también se puede leer desde un fichero utilizando la función read_corpus
stop_words=['i','me','my','myself','we','our','ours','ourselves','you','your','yours','yourself','yourselves',
            'he','him','his','himself','she','her','hers','herself','it','its','itself','they','them','their',
            'theirs','themselves','what','which','who','whom','this','that','these','those','am','is','are',
            'was','were','be','been','being','have','has','had','having','do','does','did','doing','a','an',
            'the','and','but','if','or','because','as','until','while','of','at','by','for','with','about',
            'against','between','into','through','during','before','after','above','below','to','from','up',
            'down','in','out','on','off','over','under','again','further','then','once','here','there','when',
            'where','why','how','all','any','both','each','few','more','most','other','some','such','no','nor',
            'not','only','own','same','so','than','too','very','s','t','can','will','just','don','should','now', 'ever']


# obtiene un dataframe de pandas
def read_corpus(file, sep):
    return pd.read_csv(file, sep)


# muestra gráficamente la distribución de clases del conjunto de datos utilizado
def plot(corpus):
    dist = corpus.groupby(["Sentiment"]).size()
    dist = dist / dist.sum()* 100
    fig, ax = plt.subplots(figsize=(12, 8))
    sns.barplot(dist.keys(), dist.values);
    plt.show()


# función auxiliar utilizada por CountVectorizer para procesar las frases
def english_stemmer(sentence):
    stemmer = EnglishStemmer()
    analyzer = CountVectorizer(binary=False, analyzer='word', stop_words=stop_words,
                               ngram_range=(1, 1)).build_analyzer()
    return (stemmer.stem(word) for word in analyzer(sentence))


# entrena el pipeline
def fit_model(data, target, pipeline):
    pipeline.fit(data, target)


# utiliza el pipeline para predecir datos
def predict(model, data):
    return model.predict(data)


# utiliza el pipeline para obtener la probabilidad de la predicción
def predict_proba(model, data):
    return model.predict_proba(data)


# evalua el pipeline entrenado de acuerdo a una de las métricas apropiadas para un problema de clasificación
def evaluate_model(model, X, y_true):
    y_pred = predict(model, X)

    print('==== Sumario de la clasificación ==== ')
    print(classification_report(y_true, y_pred))
    
    print('Accuracy -> {:.2%}\n'.format(accuracy_score(y_true, y_pred)))
    
    if hasattr(model, 'predict_proba'):
      y_scores = predict_proba(model, X)[:,1]
      rocs = roc_auc_score(y_true, y_scores)
      #rocc = roc_curve(y_true, y_score[:,1], pos_label='positive')

      print('ROC Score ->  {:.2%}\n'.format(rocs))
      plot_roc_curve(model, X, y_true)
      #print(rocc)

    print('==== Matriz de confusión ==== ')
    cm = confusion_matrix(y_true, y_pred)
    display_labels = unique_labels(y_true, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=display_labels)
    disp.plot(include_values=True)



# guarda un pipeline entrenado
def saveModel(model, modelName = "pickle_model.pkl"):
   pkl_filename = modelName
   with open(pkl_filename, 'wb') as file:
    pickle.dump(model, file)   


# carga un pipeline entrenado y guardado previamente
def loadModel(rutaModelo = "pickle_model.pkl"):
  # Load from file
  with open(rutaModelo, 'rb') as file:
    pickle_model = pickle.load(file)
    return pickle_model 

print('Done!')

### Creación de un pipeline para la clasificación de textos.

Para construir el pipeline, utilizaremos la clase Pipeline de sklean. Esta permite encadenar los diferentes pasos, por ejemplo, algoritmos de extracción de características y un clasificador. Por ejemplo, para obtener un pipeline que comprende CountVectorizer, seguido de TfidfTransformer y un Support Vector Machine como clasificador, se utilizaría esta sentencia:

~~~ 
Pipeline([
        ('dataVect', CountVectorizer(analyzer=english_stemmer)),
        ('tfidf', TfidfTransformer(smooth_idf=True, use_idf=True)),
        (classifier, SVC(probability=True) )
     ])
~~~

Para tener mayor flexibilidad si se desean probar varios clasificadores, podría construirse el pipeline sin clasificador, incluyendo este con posterioridad. Este será el enfoque que seguiremos en la actividad.

Ejecute la siguiente casilla para definir una función que construye un pipeline con las características antes mencionadas.


In [ ]:
def preprocessPipeLine():
    return Pipeline([
        ('dataVect', CountVectorizer(analyzer=english_stemmer)),
        ('tfidf', TfidfTransformer(smooth_idf=True, use_idf=True)),
     ])
    
print('Done!')

### Carga de datos y análisis exploratorio.

Antes de entrenar el pipeline, es necesario cargar los datos. Existen diferentes opciones, entre estas:

- montar nuestra partición de Google Drive y leer un fichero desde esta.

- leer los datos desde un fichero en una carpeta local.

- leer los datos directamente de un URL.

Ejecute la siguiente casilla prestando atención a las instrucciones adicionales en los comentarios.


In [ ]:
# descomente las siguientes 3 líneas para leer datos desde Google Drive,sumiendo que se trata de un fichero llamado review.csv localizado dentro de una carpeta llamada 'Datos' en su Google Drive
#from google.colab import drive
#drive.mount('/content/drive')
#path = '/content/drive/MyDrive/Datos/ejemplo_review_train.csv'


# descomente la siguiente línea para leer los datos desde un archivo local, por ejemplo, asumiendo que se encuentra dentro de un directorio llamado sample_data
#path = './sample_data/ejemplo_review_train.csv'


# descomente la siguiente línea para leer datos desde un URL
path = 'https://github.com/TeachingTextMining/TextClassification/raw/main/01-SA-Pipeline-Reviews/sample_data/ejemplo_review_train.csv'


# leer los datos
data = pd.read_csv(path, sep=',')

print('Done!')

Una vez leídos los datos, ejecute la siguiente casilla para construir una gráfica que muestra la distribución de clases en el corpus. 

In [ ]:
# obtener algunas estadísticas sobre los datos
categories = sorted(data['Sentiment'].unique(), reverse=True)
hist= Counter(data['Sentiment']) 
print('Total de instancias -> {0}'.format(data.shape[0]))
print('Distribución de clases -> {0}'.format({item[0]:round(item[1]/len(data['Sentiment']), 3) for item in sorted(hist.items(), key=lambda x: x[0])}))

print('Categorías -> {0}'.format(categories))
print('Comentario de ejemplo -> {0}'.format(data['Phrase'][0]))
print('Categoría del comentario -> {0}'.format(data['Sentiment'][0]))

colors = ['darkgreen', 'red']
fig = go.Figure(layout=go.Layout(height=400, width=600))
fig.add_trace(go.Bar(x=categories, y=[hist[cat] for cat in sorted(hist.keys())], marker_color=colors))
fig.show()

print('Done!')

### Entrenamiento del modelo

Ejecute la siguiente casilla  que integra todas las funciones definidas para constuir el pipeline, entrenarlo y guardarlo para su posterior uso.


In [ ]:
# crear el pipeline (solo incluyendo los pasos de pre-procesamiento)
pipeline=preprocessPipeLine()

# crear el clasificador y añadirlo al pipeline. Puede probar diferentes clasificadores
# classifier = MultinomialNB()
# classifier = DecisionTreeClassifier()
classifier = SVC(probability=True)

pipeline.steps.append(('classifier', classifier))


# obtener conjuntos de entrenamiento (90%) y validación (10%)
seed = 0    # fijar random_state para reproducibilidad
train, val = train_test_split(data, test_size=.1, stratify=data['Sentiment'], random_state=seed)


# entrenar el modelo
fit_model(train['Phrase'], train['Sentiment'], pipeline)


# guardar el modelo
saveModel(pipeline)

print('Done!')

Luego de entranado el modelo, podemos evaluar su desempeño en los conjuntos de entramiento y validación.

Ejecute la siguiente casilla para evaluar el modelo en el conjunto de entrenamiento.

In [ ]:
# predecir y evaluar el modelo en el conjunto de entrenamiento
print('==== Evaluación conjunto de entrenamiento ====')

evaluate_model(model, train['Phrase'], train['Sentiment'])

print('Done!')

Ejecute la siguiente casilla para evaluar el modelo en el conjunto de validación. Compare los resultados.

In [ ]:
# predecir y evaluar el modelo en el conjunto de entrenamiento
print('==== Evaluación conjunto de validación ====')

evaluate_model(model, val['Phrase'], val['Sentiment'])

print('Done!')

In [ ]:
import numpy as np
from sklearn.metrics import roc_curve
y = np.array([1, 1, 2, 2])
scores = np.array([0.1, 0.4, 0.35, 0.8])
fpr, tpr, thresholds = roc_curve(y, scores, pos_label=2)


## Predicción de nuevos datos.

Una vez entrenado el modelo, podemos evaluar su rendimiento en datos no utilizados durante el entrenamiento o emplearlo para predecir nuevas instancias. En cualquier caso, se debe cuidar realizar los pasos de pre-procesamiento necesarios según el caso. En el ejemplo, utilizaremos la porción de prueba preparada inicialmente.

**Notar que**:
-  se cargará el modelo previamente entrenado, estableciendo las configuraciones pertinentes.


### Instanciar modelo pre-entrenado

Para predecir nuevas instancias es preciso cargar el modelo previamente entrenado. Esto dependerá del formato en el que se exportó el modelo, pero en general se requieren dos elementos: la estructura del modelo y los pesos. 

Ejecute la siguiente casilla para cargar el modelo entrenado previamente.

In [ ]:
# cargar pipeline entrenado
model = loadModel()

print('Done!')

### Predecir nuevos datos

Con el modelo cargado, es posible utilizarlo para analizar nuevos datos. 

Ejecute las siguientes casillas para:

(a) categorizar un texto de muestra.

(b) cargar nuevos datos, categorizarlos y mostrar algunas estadísticas sobre el corpus.

In [ ]:
# ejemplo de texto a clasificar
text = ['Brian De Palma\'s undeniable virtuosity can\'t really camouflage the fact that his plot here is a thinly disguised\
        \"Psycho\" carbon copy, but he does provide a genuinely terrifying climax. His "Blow Out", made the next year, was an improvement.']


# predecir los nuevos datos
pred_labels = model.predict(text)
pred_proba = model.predict_proba(text)
print('La categoría del review es -> {0}'.format(pred_labels))
print('La confidencia de la predicción es -> {0}'.format(pred_proba))

print('Done!')

También podemos predecir nuevos datos cargados desde un fichero. 

Ejecute la siguiente casilla, descomentando las instrucciones necesarias según sea el caso.

In [ ]:
# descomente las siguientes 3 líneas para leer datos desde Google Drive,sumiendo que se trata de un fichero llamado review.csv localizado dentro de una carpeta llamada 'Datos' en su Google Drive
#from google.colab import drive
#drive.mount('/content/drive')
#path = '/content/drive/MyDrive/Datos/ejemplo_review_train.csv'


# descomente la siguiente línea para leer los datos desde un archivo local, por ejemplo, asumiendo que se encuentra dentro de un directorio llamado sample_data
#path = './sample_data/ejemplo_review_train.csv'


# descomente la siguiente línea para leer datos desde un URL
path = 'https://github.com/TeachingTextMining/TextClassification/raw/main/01-SA-Pipeline-Reviews/sample_data/ejemplo_review_test.csv'


# leer los datos
new_data = pd.read_csv(path, sep=',')

print('Done!')

Ejecute la siguiente celda para predecir los datos y mostrar algunas estadísticas sobre el análisis realizado.

In [ ]:
# predecir los datos de prueba
pred_labels = model.predict(new_data['Phrase'])


# obtener algunas estadísticas sobre la predicción en el conjunto de pruebas
categories = ['positive', 'negative']
hist = Counter(pred_labels) 

colors = ['darkgreen', 'red']
fig = go.Figure(layout=go.Layout(height=400, width=600))
fig.add_trace(go.Bar(x=categories, y=[hist[cat] for cat in sorted(hist.keys())], marker_color=colors))
fig.show()

print('Done!')